<a href="https://colab.research.google.com/github/jkranyak/project_3/blob/main/project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Apr 10 20:29:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install tensorflow


In [4]:
!pip install isic-cli


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.1 MB/s eta 0:00:00


In [5]:
! isic user login

visit the following url in a browser:
https://api.isic-archive.com/oauth/authorize?response_type=code&client_id=RpCzc4hFjv5gOJdM2DM2nBdokOviOh5ne63Tpn7Q&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=kmGwfoz4avuEVKECGVadANUQxs5Q1D&code_challenge=wNZcxvZUgcaMlKSwsGU8BhZt_yqDd7JTA6dVzC5JrW0&code_challenge_method=S256
enter the code shown in your browser: 8feAFBcQvyoeNl9BDi5Kjx00Or6x3D
Success!


Step 2: Query the Dataset
Endpoints: Use the API endpoints to query the dataset. Common operations include listing available images, retrieving image metadata, and downloading images.
Filtering: Utilize query parameters to filter the dataset based on your criteria, such as diagnosis, image type, or other metadata.

In [6]:
!pip install isic-cli


In [7]:
! pip install kaggle

In [8]:
!isic collection list


┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ ID  ┃ Name                                          ┃ Public ┃ Pinned ┃ Locked ┃ DOI             ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 249 │ BCN20000                                      │ True   │ False  │ False  │ None            │
│ 61  │ Challenge 2016: Test                          │ True   │ True   │ True   │ None            │
│ 74  │ Challenge 2016: Training                      │ True   │ True   │ True   │ None            │
│ 69  │ Challenge 2017: Test                          │ True   │ True   │ True   │ None            │
│ 60  │ Challenge 2017: Training                      │ True   │ True   │ True   │ None            │
│ 71  │ Challenge 2017: Validation                    │ True   │ True   │ True   │ None            │
│ 64  │ Challenge 2018: Task 1-2: Test                │ True   │ True   │ True   │ None    

In [9]:
import requests
import os
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import EfficientNetB0

from google.colab import drive
from PIL import Image

In [10]:
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


Comment out if already running

In [11]:
# Specify the current path of kaggle.json file
current_path = '/content/kaggle.json'

# Desired path where the Kaggle API expects the kaggle.json file
desired_path = '/root/.kaggle/kaggle.json'

if os.path.exists(current_path):
    os.makedirs(os.path.dirname(desired_path), exist_ok=True)
    os.rename(current_path, desired_path)

    # Set the file's permissions to avoid a permissions error
    os.chmod(desired_path, 0o600)
else:
    print(f"Error: '{current_path}' does not exist. Please upload the file.")



find the data set on kaggle

Comment out if running and already have downloaded images

In [12]:
!kaggle datasets download -d andrewmvd/isic-2019


100% 9.09G/9.10G [01:42<00:00, 121MB/s] 
100% 9.10G/9.10G [01:42<00:00, 95.6MB/s]


In [ ]:
!unzip -q isic-2019.zip


Step 1: Data Preparation and Preprocessing
DICOM Images

DICOM Handling: Use pydicom to read DICOM files, which contain both the images and embedded metadata. For each DICOM file, extract the image data for preprocessing and the metadata for analysis.
Image Preprocessing: Since DICOM images might not be uniformly sized, you may need to resize them to a standard size (e.g., 1024x1024) to match the TFRecord images, normalize pixel values, and potentially augment the data to improve model robustness.
JPEG and TFRecord Images

JPEG Handling: Directly read and preprocess (resize if necessary, normalize, augment).
TFRecord Handling: Extract and preprocess. TFRecord files are already uniformly sized, which simplifies preprocessing.
Metadata

CSV Metadata: Load the metadata from CSV files. Clean and preprocess this data, which involves handling missing values, normalizing or scaling numerical features, and encoding categorical features.



Load the metadata

and

Explore the Metadata CSV: Load the metadata.csv files for training, test, and validation sets to understand the structure and types of data available. This step is crucial for preprocessing and feature selection.

In [ ]:
# Load the metadata
metadata = pd.read_csv('/content/ISIC_2019_Training_Metadata.csv')

# Display the first few rows of each DataFrame
metadata.head()


In [ ]:
ground_truth = pd.read_csv('/content/ISIC_2019_Training_GroundTruth.csv')
ground_truth

In [ ]:
# Load the CSV files
ground_truth = pd.read_csv('/content/ISIC_2019_Training_GroundTruth.csv')
metadata = pd.read_csv('/content/ISIC_2019_Training_Metadata.csv')

image_dir = '/content/ISIC_2019_Training_Images'
ground_truth['image_path'] = ground_truth['image'].apply(lambda x: os.path.join(image_dir, x + '.jpg'))

# Merge the ground_truth with metadata if necessary
full_metadata = pd.merge(ground_truth, metadata, on='image', how='left')  # Adjust 'on' parameter as needed
full_metadata

In [ ]:
# Display the image paths for the first few entries
print(full_metadata['image_path'].head())


In [ ]:
# Correct the base path in 'image_path' column
correct_base_path = "/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input"

full_metadata['image_path'] = full_metadata['image'].apply(lambda x: f"{correct_base_path}/{x}.jpg")

# Verify the correction by printing the first few entries again
print(full_metadata['image_path'].head())


In [ ]:
full_metadata.info()

In [ ]:
# Fill missing values for 'age_approx' with its median
full_metadata['age_approx'].fillna(full_metadata['age_approx'].median(), inplace=True)

# For categorical data, fill missing values with 'unknown'
full_metadata['anatom_site_general'].fillna('unknown', inplace=True)
full_metadata['sex'].fillna('unknown', inplace=True)


In [ ]:
full_metadata = pd.get_dummies(full_metadata, columns=['anatom_site_general', 'sex'])


In [ ]:
# Let's separate features and labels first
X = full_metadata.drop(['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'], axis=1)
y = full_metadata[['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']]

# Now, we split the data into training and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the X_temp and y_temp further into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)


Dual Input Generator for creating a mo

In [ ]:
class DualInputGenerator(Sequence):
    def __init__(self, image_paths, metadata, labels, batch_size, img_size=(224, 224), shuffle=True):
        self.image_paths = image_paths
        self.metadata = metadata
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Initialization
        X_images = np.empty((self.batch_size, *self.img_size, 3))
        X_metadata = np.empty((self.batch_size, self.metadata.shape[1]))
        y = np.empty((self.batch_size, self.labels.shape[1]), dtype=int)

        # Generate data
        for i, batch_index in enumerate(batch_indexes):
            # Load and preprocess image
            img_path = os.path.join('/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input/', self.image_paths[batch_index])
            img = load_img(img_path, target_size=self.img_size)
            X_images[i,] = img_to_array(img) / 255.0

            # Load metadata
            X_metadata[i,] = self.metadata[batch_index]

            # Load label
            y[i,] = self.labels[batch_index]

        return [X_images, X_metadata], y


    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


In [ ]:
# Initialize the training generator
train_gen = DualInputGenerator(
    image_paths=X_train['image_path'].values,
    metadata=X_train.drop(columns=['image', 'image_path', 'lesion_id']).values,
    labels=y_train.values,  # Include labels for training set
    batch_size=32
)

# Initialize the validation generator
val_gen = DualInputGenerator(
    image_paths=X_val['image_path'].values,
    metadata=X_val.drop(columns=['image', 'image_path', 'lesion_id']).values,
    labels=y_val.values,  # Include labels for validation set
    batch_size=32
)

# Initialize the test generator
test_gen = DualInputGenerator(
    image_paths=X_test['image_path'].values,
    metadata=X_test.drop(columns=['image', 'image_path', 'lesion_id']).values,
    labels=y_test.values,  # Include labels for test set
    batch_size=32
)



define a dual input model


In [ ]:
num_metadata_features = X_train.drop(columns=['image', 'image_path', 'lesion_id']).shape[1]
print("Number of metadata features:", num_metadata_features)


In [ ]:
# Load EfficientNetB0 as the base model
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze EfficientNet model

# Image processing branch
image_input = base_model.input
x = base_model(image_input)
x = GlobalAveragePooling2D()(x)
image_branch = Dense(128, activation='relu')(x)

# Metadata input branch
metadata_input = Input(shape=(13,), name='metadata_input')  # num_metadata_features defined based on your dataset
y = Dense(32, activation='relu')(metadata_input)
metadata_branch = Dense(64, activation='relu')(y)

# Combine branches
number_of_classes = y_train.shape[1]
combined = concatenate([image_branch, metadata_branch])
z = Dense(256, activation='relu')(combined)
z = Dropout(0.5)(z)
output = Dense(number_of_classes, activation='softmax')(z)  # number_of_classes defined based on your dataset

# Final model
model = Model(inputs=[image_input, metadata_input], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


instal a checkpoint for this, since the rentime exceeds several hours

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = "/content/drive/My Drive/your_model_directory/model-{epoch:02d}-{val_accuracy:.2f}.hdf5"

checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=3, save_best_only=True, mode='max')

Juice it UP!

In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
tf.debugging.set_log_device_placement(True)

# Test with a simple computation
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
# Run on GPU
c = tf.matmul(a, b)

print(c)


First Run without pretrained generators

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen)
)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_gen, steps=len(test_gen))
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Lets save this model


In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

# Generate a plot of the model
plot_model(model, to_file='model_diagram.png', show_shapes=True, show_layer_names=True)


In [ ]:
from IPython.display import Image

# Display the model diagram
Image('model_diagram.png')


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = "/content/drive/My Drive/your_model_directory/model-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


66.46% is ok, but lets do some fine tuning

In [ ]:
# Unfreeze some or all of the base model layers
base_model.trainable = True

# It's important to recompile the model after making any changes to the `trainable` attribute of any layer
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5  # Adjust the number of epochs for fine-tuning
)


In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Step 5: User Interface (UI)
A simple web-based UI allows users to upload lesion images, input relevant metadata, and receive a prediction. We will use Gradio as learned in class for this model.

In [ ]:
! pip install gradio

In [ ]:
import gradio as gr
from tensorflow.keras.preprocessing import image
import numpy as np


In [ ]:
def predict_image(img):
    # Assuming 'model' is your trained dual-input model
    # Preprocess the image to match your model's expected input
    img = img.resize((224, 224))  # Resize image
    img_array = image.img_to_array(img) / 255.0  # Convert to array and normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Example: Placeholder for metadata, replace with actual expected metadata
    metadata = np.zeros((1, num_metadata_features))  # You'll need to replace this with actual metadata handling

    # Make prediction
    prediction = model.predict([img_array, metadata])  # Dual input: image and metadata

    # Assuming you have a function to process your model's output into a human-readable form
    readable_prediction = process_prediction(prediction)  # Implement this function based on your model's output

    return readable_prediction

# Create a Gradio interface
iface = gr.Interface(fn=predict_image,
                     inputs=gr.inputs.Image(shape=(224, 224)),
                     outputs="text",
                     title="Skin Lesion Classifier",
                     description="Upload an image of a skin lesion to predict its type.")

# Launch the app
iface.launch()


we will want to enhance image upload by filling out as much metadata as possible on the input side

In [ ]:
# Define a prediction function that includes metadata inputs
def predict_image_with_metadata(img, age, sex, location):
    # Preprocess inputs as needed
    # For example, you would need to convert 'sex' and 'location' into the format your model expects (e.g., one-hot encoded)

    # Make prediction using preprocessed image and metadata
    # prediction = model.predict(...)

    # Convert prediction to readable format
    return process_prediction(prediction)

# Create Gradio interface with additional metadata inputs
iface = gr.Interface(fn=predict_image_with_metadata,
                     inputs=[gr.inputs.Image(shape=(224, 224)),
                             gr.inputs.Number(label="Age"),
                             gr.inputs.Radio(['Male', 'Female', 'Other'], label="Sex"),
                             gr.inputs.Dropdown(['Head/Neck', 'Upper Extremity', 'Lower Extremity', 'Torso', 'Palms/Soles', 'Oral/Genital', 'Other'], label="Anatomical Site")],
                     outputs="text",
                     title="Skin Lesion Classifier",
                     description="Upload an image of a skin lesion and enter metadata to predict its type.")

iface.launch()


we will need to preprocess the uploaded imagages

In [ ]:
from PIL import Image
import numpy as np

def preprocess_image(uploaded_image):
    """
    Preprocess the uploaded image to match the model's expected input format.
    """
    # Resize the image to the required dimensions
    img = uploaded_image.resize((224, 224))

    # Convert the image to a numpy array
    img_array = np.array(img)

    # Ensure the image has 3 color channels (RGB)
    if img_array.ndim == 2:
        img_array = np.stack((img_array,)*3, axis=-1)

    # Normalize pixel values to [0, 1]
    img_array = img_array / 255.0

    # Add a batch dimension
    img_array = np.expand_dims(img_array, axis=0)

    return img_array

def predict_image(uploaded_image):
    # Preprocess the uploaded image
    preprocessed_image = preprocess_image(uploaded_image)

    # Assuming 'model' is your trained model
    prediction = model.predict(preprocessed_image)

    # Process the prediction to generate a human-readable result
    readable_prediction = process_prediction(prediction)  # Implement this based on your model's specifics

    return readable_prediction


ok, lets finish the workflow by making a prediction on the preprocessed image

In [ ]:
diagnosis_descriptions = {
    'MEL': 'Melanoma: a serious form of skin cancer that begins in cells known as melanocytes.',
    'NV': 'Melanocytic nevus: a common type of skin growth that often appears as a small, dark brown spot.',
    'BCC': 'Basal cell carcinoma: a type of skin cancer that most often develops on areas exposed to the sun.',
    'AK': 'Actinic keratosis: a rough, scaly patch on the skin caused by years of sun exposure.',
    'BKL': 'Benign keratosis: a non-cancerous skin condition that appears as a waxy brown, black, or tan growth.',
    'DF': 'Dermatofibroma: a common growth on the skin, usually found on the lower legs, that can be pink, red, or brown.',
    'VASC': 'Vascular lesion: a type of abnormal growth or mark on the skin that is made up of blood vessels.',
    'SCC': 'Squamous cell carcinoma: a common form of skin cancer that develops in the squamous cells.',
    'UNK': 'None of the others: the lesion does not fit into any of the other categories.'
}


In [ ]:
def predict_and_explain(image):
    preprocessed_image = preprocess_image(image)
    prediction = model.predict(preprocessed_image)
    predicted_class_index = np.argmax(prediction, axis=1)[0]

    # Assuming your model's output classes are ordered as listed in diagnosis_descriptions
    class_keys = list(diagnosis_descriptions.keys())
    predicted_class_key = class_keys[predicted_class_index]

    # Fetch the description
    description = diagnosis_descriptions[predicted_class_key]

    return predicted_class_key, description



and then display the prediction

In [ ]:
def gradio_predict(image):
    class_key, description = predict_and_explain(image)
    return f"Diagnosis: {class_key} ({description})"

iface = gr.Interface(fn=gradio_predict,
                     inputs=gr.inputs.Image(shape=(224, 224)),
                     outputs="text",
                     title="Skin Lesion Classifier",
                     description="Upload an image of a skin lesion to predict its type and get more information.")
iface.launch()



4. Integrate APIs

If using OpenAI's GPT-3, you'll interact with it through its API, sending requests with the classification results and receiving generated text.
If your application needs to store or retrieve data, consider integrating a database and accessing it through a REST API.


5. Build the Application Logic

Develop a user interface where users can upload photos of skin growths.
After uploading, the photo is sent to the image classification model to determine the type of growth.
The classification result is used to generate a prompt for the ChatGPT-like model, which then creates an informative response.
Display the response to the user along with any additional advice or recommendations.